In [2]:
from opensearchpy import OpenSearch

host = 'localhost'
port = 9200
auth = ('admin', 'YourStrongPassword123!') # For testing only. Don't store credentials in code.
ca_certs_path = '/full/path/to/root-ca.pem' # Provide a CA bundle if you use intermediate CAs with your root CA.

# Optional client certificates if you don't want to use HTTP basic authentication.
# client_cert_path = '/full/path/to/client.pem'
# client_key_path = '/full/path/to/client-key.pem'

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    # client_cert = client_cert_path,
    # client_key = client_key_path,
    use_ssl = False,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    # ca_certs = ca_certs_path
)

# Create an index with non-default settings.
index_name = 'python-test-index'
index_body = {
  'settings': {
    'index': {
      'number_of_shards': 4
    }
  }
}

response = client.indices.create(index=index_name, body=index_body)
print('\nCreating index:')
print(response)

# Add a document to the index.
document = {
  'title': 'Moneyball',
  'director': 'Bennett Miller',
  'year': '2011'
}
id = '1'

response = client.index(
    index = index_name,
    body = document,
    id = id,
    refresh = True
)

print('\nAdding document:')
print(response)

# Perform bulk operations

movies = '{ "index" : { "_index" : "my-dsl-index", "_id" : "2" } } \n { "title" : "Interstellar", "director" : "Christopher Nolan", "year" : "2014"} \n { "create" : { "_index" : "my-dsl-index", "_id" : "3" } } \n { "title" : "Star Trek Beyond", "director" : "Justin Lin", "year" : "2015"} \n { "update" : {"_id" : "3", "_index" : "my-dsl-index" } } \n { "doc" : {"year" : "2016"} }'

client.bulk(body=movies)

# Search for the document.
q = 'miller'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'director']
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results:')
print(response)

# Delete the document.
response = client.delete(
    index = index_name,
    id = id
)

print('\nDeleting document:')
print(response)

# Delete the index.
response = client.indices.delete(
    index = index_name
)

print('\nDeleting index:')
print(response)



Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'python-test-index'}

Adding document:
{'_index': 'python-test-index', '_id': '1', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}

Search results:
{'took': 29, 'timed_out': False, '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.13076457, 'hits': [{'_index': 'python-test-index', '_id': '1', '_score': 0.13076457, '_source': {'title': 'Moneyball', 'director': 'Bennett Miller', 'year': '2011'}}]}}

Deleting document:
{'_index': 'python-test-index', '_id': '1', '_version': 2, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}

Deleting index:
{'acknowledged': True}


** model register

In [2]:
import time
from opensearchpy import OpenSearch

# Cấu hình kết nối (thay đổi theo môi trường của bạn)
client = OpenSearch(
    hosts=[{'host': 'localhost', 'port': 9200}],
    http_auth=('admin', 'YourStrongPassword123!'),
    use_ssl=False,
    verify_certs=False
)

def wait_for_task(task_id):
    """Đợi cho đến khi task hoàn thành và trả về kết quả."""
    while True:
        status = client.transport.perform_request('GET', f'/_plugins/_ml/tasks/{task_id}')
        state = status['state']
        if state == 'COMPLETED':
            return status
        elif state == 'FAILED':
            raise Exception(f"Task failed: {status}")
        print(f"Đang xử lý... Trạng thái hiện tại: {state}")
        time.sleep(2)

# --- BƯỚC 1: Đăng ký Model ---
print("--- Đang đăng ký model ---")
model_config = {
    "name": "huggingface/sentence-transformers/all-MiniLM-L6-v2",
    "version": "1.0.1",
    "model_format": "TORCH_SCRIPT"
}

# Đăng ký model và nhận task_id
register_response = client.transport.perform_request(
    'POST', '/_plugins/_ml/models/_register', body=model_config
)
task_id = register_response['task_id']
print(f"Task Register ID: {task_id}")

# Đợi task đăng ký xong để lấy model_id
task_result = wait_for_task(task_id)
model_id = task_result['model_id']
print(f"Model ID của bạn là: {model_id}")

# --- BƯỚC 2: Deploy Model ---
print("\n--- Đang deploy model ---")
deploy_response = client.transport.perform_request(
    'POST', f'/_plugins/_ml/models/{model_id}/_deploy'
)
deploy_task_id = deploy_response['task_id']

# Đợi deploy xong
wait_for_task(deploy_task_id)
print(f"Model {model_id} đã sẵn sàng sử dụng!")

# --- BƯỚC 3: Kiểm tra ---
check_model = client.transport.perform_request('GET', f'/_plugins/_ml/models/{model_id}')
print("\nThông tin model hiện tại:")
print(check_model)

--- Đang đăng ký model ---
Task Register ID: OohpRpwBgZqRe2XB5aRA
Đang xử lý... Trạng thái hiện tại: CREATED
Đang xử lý... Trạng thái hiện tại: CREATED
Đang xử lý... Trạng thái hiện tại: CREATED
Đang xử lý... Trạng thái hiện tại: CREATED
Đang xử lý... Trạng thái hiện tại: CREATED
Đang xử lý... Trạng thái hiện tại: CREATED
Đang xử lý... Trạng thái hiện tại: CREATED
Đang xử lý... Trạng thái hiện tại: CREATED
Đang xử lý... Trạng thái hiện tại: CREATED
Đang xử lý... Trạng thái hiện tại: CREATED
Model ID của bạn là: O4hpRpwBgZqRe2XB7KT7

--- Đang deploy model ---
Đang xử lý... Trạng thái hiện tại: CREATED
Model O4hpRpwBgZqRe2XB7KT7 đã sẵn sàng sử dụng!

Thông tin model hiện tại:
{'name': 'huggingface/sentence-transformers/all-MiniLM-L6-v2', 'model_group_id': 'NohHRpwBgZqRe2XBDKQ_', 'algorithm': 'TEXT_EMBEDDING', 'model_version': '2', 'model_format': 'TORCH_SCRIPT', 'model_state': 'DEPLOYED', 'model_content_size_in_bytes': 91790008, 'model_content_hash_value': 'c15f0d2e62d872be5b5bc6c84d2e0f